In [1]:
#!pip install psycopg2

In [2]:
#!pip install python-dotenv

In [3]:
#!pip install touch

# Create Temp Table and Join Meta Data with GitHub Contributor and Commits Data

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import os
import sys
#https://stackoverflow.com/questions/40216311/reading-in-environment-variables-from-an-environment-file
from dotenv import load_dotenv
load_dotenv()
import touch
import time
pd.options.display.max_columns = None

In [5]:
os.getcwd()

'/sfs/qumulo/qproject/class/sds_sdad'

In [6]:
#create env file goto home dir and add user and password only once
#touch.touch('.env')

### Config params for connection and test connection

In [2]:
#making conenction to postgresSQL DB
pgpassword = os.getenv('password')
pguser = os.getenv('user')
host =  'postgis1-s.bii.virginia.edu'
database = 'sdad'
#port = 

#trying connection
dbserver_p = psycopg2.connect(
    user=pguser, 
    password=pgpassword, 
    host=host,
    database=database 
)

#dbserver_p.autocommit = True
cursor_p = dbserver_p.cursor()


### Connect to the database and create engine

In [3]:
engine_p = create_engine("postgresql+psycopg2://{user}:{pw}@{host}/{db}"
                       .format(user=pguser, pw=pgpassword, host=host, db=database))

In [4]:
# Take in a PostgreSQL engine and query and return dataframe
def load_db_table(query, eng):
    '''
    Function which will use connection engine with pd.read_sql_query
    '''
    data = pd.read_sql_query(query, con=eng)
    return data

In [5]:
def load_tbl_cur(query, cursor):
    '''
    Function which will use cursor and fetch all the rows
    from result of sql query and convert to pandas dataframe
    '''
    cursor.execute(query)
    df = cursor.fetchall()
    colnames = [x[0] for x in cursor.description]
    return pd.DataFrame(df, columns=colnames)

### Query

### Method 1

In [6]:
myquery = """
         SELECT * FROM gh.pypi_slug_contrib_commits LIMIT 10;
"""
df_tbl = load_db_table(myquery,engine_p)

In [7]:
df_tbl

,name,version,author,author_email,license,home_page,slug,committed_date,login,additions,deletions,email,company,organization,location,country,is_us,is_null,is_academic,academic_frac,us_academic,frgn_academic,is_business,business_frac,us_business,frgn_business,is_gov,gov_frac,us_gov,frgn_gov,is_nonprofit,nonprofit_frac,us_nonprofit,frgn_nonprofit,is_household,household_frac,us_household,frgn_household,na_fraction,frgn_fraction,us_fraction
0,flask-websub,0.1,marten de vries,m@rtendevri.es,isc,https://github.com//,/,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,starter.templates.python-django,0.1.2,kirill klenov,horneds@gmail.com,bsd,http://github.com//,/,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,starter.templates.python-django,0.1.0,kirill klenov,horneds@gmail.com,bsd,http://github.com//,/,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,starter.templates.python-django,0.1.1,kirill klenov,horneds@gmail.com,bsd,http://github.com//,/,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,msd-regression-models,0.0.1,...,oumaima.taheri@um6p.ma,bsd-3,https://github.com/..,..,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,potrans,0.1.2,mikhail rubin,mihanentalpo@yandex.ru,mit,https://github.com/...,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,stallings-graphs,0.1,pascal weil,pascal.weil@labri.fr,gplv2+,https://github.com/...,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,mitcf,0.1,florian metzler,fmetzler@mit.edu,mit,https://github.com/...,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,jira-power-tools,0.1.0,david bradford,david.bradford@mongodb.com,apache-2.0,https://github.com/...,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
9,myownlib,0.0.2,me,me@email.com,mit,https://github.com/...,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


### Method 2

In [22]:
myquery1 = """
        select count(*) 
        from gh.commits_dd_nmrc_jbsc 
        

"""
df_tbl1 = load_db_table(myquery1,engine_p)

In [23]:
df_tbl1 

,count
0,312955618


### Join Data Together 

Goal is to left join meta data with gh.commits_dd_nmrc_jbsc on the slug and another left join with gh_cost.sectored_fractioned_103121 on login

### Files created by Brandon

In [10]:
# read in meta file to create temp table
tempdf = pd.read_csv('/project/class/sds_sdad/oss_capstone2021-2022/clean_data_11132021/pypi_meta_with_license_github_slugs.csv')
tempdf = tempdf.drop('Unnamed: 0')

/apps/software/standard/compiler/gcc/9.2.0/jupyter_conda/2020.11-py3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
tempdf.shape

(10687591, 9)

In [25]:
# create copy of tempdf - this dataframe will be uploaded to the database as a temp file
slug_df_copy = tempdf #[['slug']]
slug_df_copy

,name,version,author,author_email,license,home_page,dependency,slug
0,vcert,0.6.6,denis subbotin,denis.subbotin@venafi.com,asl,https://github.com/venafi/vcert-python,certvalidator,venafi/vcert-python
1,cfblog,0.1.4,vinay karanam,vinayinvicible@gmail.com,bsd,https://github.com/coverfox/cfblog/,NaN,coverfox/cfblog/
2,hostblock,0.1.1,predrag mandic,info@nul.one,bsd,https://github.com/nul-one/hostblock,NaN,nul-one/hostblock
3,fast-histogram,0.8,thomas robitaille,thomas.robitaille@gmail.com,bsd,https://github.com/astrofrog/fast-histogram,numpy,astrofrog/fast-histogram
4,poxls,1.2.0,wichert akkerman,wichert@wiggy.net,bsd,https://github.com/wichert/po-xls,NaN,wichert/po-xls
...,...,...,...,...,...,...,...,...
10687586,odoo14-addon-sale-tier-validation,14.0.1.0.1.dev4,"open source integrators, odoo community associ...",support@odoo-community.org,agpl-3,https://github.com/oca/sale-workflow,NaN,oca/sale-workflow
10687587,tfds-nightly,4.4.0.dev202111120107,google inc.,packages@tensorflow.org,apache 2.0,https://github.com/tensorflow/datasets,NaN,tensorflow/datasets
10687588,tfds-nightly,4.4.0.dev202111120107,google inc.,packages@tensorflow.org,apache 2.0,https://github.com/tensorflow/datasets,NaN,tensorflow/datasets
10687589,tfds-nightly,4.4.0.dev202111130106,google inc.,packages@tensorflow.org,apache 2.0,https://github.com/tensorflow/datasets,NaN,tensorflow/datasets


In [35]:
%%time
# create temp table 
slug_df_copy.to_sql('temp_meta',schema='gh',con=engine_p, index=False, chunksize=1000, if_exists='replace')

CPU times: user 3min 18s, sys: 1min 21s, total: 4min 40s
Wall time: 24min 46s


In [ ]:
%%time
# joins on joins on joins 
myquery1 = """
        select *
        from gh.temp_meta as a
        left join gh.commits_dd_nmrc_jbsc as b ON b.slug = a.slug
        left join gh_cost.sectored_fractioned_103121 as c ON b.login = c.login
        limit 10000000

"""
df_join1 = load_db_table(myquery1,engine_p)
df_join1


In [ ]:
df_join1.to_csv('meta_left_joined_ghcommits_dd_nmrc_jbsc_on_slug_left_joined_gh_costsectored_fractioned_103121_on_login_11-14-21.csv', sep = '\t')

### Close conenction

In [15]:
# Close the connection
dbserver_p.commit()
dbserver_p.close()

In [18]:
com_df = tempdf.groupby('name')['n_slash'].sum()

In [19]:
com_df = com_df.reset_index()

In [20]:
com_df[com_df.n_slash]

KeyError: "None of [Int64Index([  9,  36, 140,  56, 286, 104,  46, 469, 808,   4,\n            ...\n             15,  17,  10,   2, 184, 216,   6, 285,   9,   6],\n           dtype='int64', length=74252)] are in the [columns]"

In [ ]:
counts = pd.read_csv('/project/class/sds_sdad/gh_pypi_counts_1121.csv')

In [ ]:
len(tempdf.slug.unique())

In [ ]:
counts.shape

In [ ]:
gh = pd.read_csv('/project/class/sds_sdad/oss_capstone2021-2022/pypi_meta_with_license_dependecies_github.csv')

In [ ]:
len(gh.home_page.unique())

In [ ]:
gh[gh.name.isin(['numpy'])]

In [ ]:
gh.query('name.str.contains("numpy")',engine='python')

In [ ]:
gh[['name','version','home_page']].drop_duplicates()

In [ ]:
gh[gh.name=='transformers'][['name','version','home_page']].drop_duplicates()

In [ ]:
# SKIP
# create temp table ... of meta data 
'''
slug_join_commits_q = r"""
        select slug
        from gh.commits_dd_nmrc_jbsc as gh
        left join ({0}) AS frame(slug) ON gh.slug = frame.slug
        Limit 10

""".format(slug_df_copy.values)

df_tbl = pd.read_sql(slug_join_commits_q,engine_p)
'''

